In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
import scipy
#%matplotlib inline
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from scipy.stats import entropy

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [0]:


def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 10))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

#dim is the full output shape of predictions for all points for all dropout masks
def create_dropout_masks(dim, num_masks, drop_prob):
    """Creates dropout masks that are fixed across all model inputs for use in estimation of Var(Y_{sample})
    Note: this function is specific to keras models used in this experiment
    #Arguments
        dim: the numpy array shape of values entering the generated dropout masks for all points across all J fixed dropout masks
        num_masks: the number of fixed dropout masks to be created (J)
        drop_prob: the dropout probability
    #Returns
        dropout masks used in estimation of Var(p_{sample})
    """
    if (len(dim)==4):
        return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3])), p=[drop_prob, 1-drop_prob])
    return 1/(1-drop_prob)*np.random.choice(2, size=((num_masks, 1, dim[2], dim[3], dim[4])), p=[drop_prob, 1-drop_prob])

def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
    return np.array(layer_output)

def fixed_mask_forward_pass (model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks):
      """Makes model predictions with J dropout masks that are fixed across points to enable estimation of Var(p_{sample})
    Function is specific to the given Keras model.
    #Arguments
        model: keras model
        forward_pass_input: X_{sample}
        num_masks: J, the number of dropout masks being used in estimation of Var(p_{sample}) and calculation of the EI acquisition function
        dropout_prob: dropout probability 
        conv_masks: first set of dropout masks, applied after MaxPooling2D layer
        dense_masks: second set of dropout masks, applied after the first Dense layer
    #Returns
        MC dropout predictions across sample points enabling estimation of Var(p_{sample}), i.e. \hat{p}_{sample}
    """
    conv = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[4].output])

    dense_1 = K.function([model.layers[6].input, K.learning_phase()],
                   [model.layers[8].output])

    dense_2 = K.function([model.layers[10].input, K.learning_phase()],
                   [model.layers[11].output])
    arr_input = np.array(forward_pass_input)
    num_points = arr_input.shape[0]
    if len(arr_input.shape) == 3:
      forward_pass_input = np.expand_dims(forward_pass_input, axis=1)
    conv_output = np.array(conv((forward_pass_input, 1)))
    dense_1_input = apply_dropout_masks(conv_output, conv_masks)
    if num_points == 1:
        dense_1_input = np.expand_dims(dense_1_input, axis=1)
    dense_1_output = multi_mask_predict(dense_1, dense_1_input)
    dense_2_input = apply_dropout_masks(dense_1_output, dense_masks)
    if num_points == 1:
        dense_2_input = np.expand_dims(dense_2_input, axis=1)
    dense_2_output = multi_mask_predict(dense_2, dense_2_input)
    return np.squeeze(dense_2_output)

def run_model (train_data_indices):
  """Trains and Keras model with the training points specified by train_data_indices and evaluates model on test data
  #Arguments:
      train_data_indices: indices of current training points within X_train_All
  #Returns:
      Test accuracy and trained Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train = keras.utils.to_categorical(y_train, num_classes=10)
  train_size = y_train.shape[0]
  Weight_Decay = 0.01/train_size
  dropout_prob = 0.25
  nb_filters = 40
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 10
  model = Sequential()
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, kernel_size=nb_conv, strides=2, data_format = "channels_first"))
  model.add(Activation('relu'))
	
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))

  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input,K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), model]

def batch_bald (fixed_mask_forward_pass_results, num_classes, num_masks, batchbald_mc_samples, batch_size):
    """Runs BatchBald, greedily assembling a queried batch of points. 
    #Arguments
      fixed_mask_forward_pass_results: Fixed-mask predictions across candidates, i.e. \hat{p}_sample (shape: [num_masks, num_points, num_classes])
      num_classes: number of total classes
      num_masks: J, the number of fixed dropout masks used in making predictions on X_cand
      batchbald_mc_samples: number of MC samples from joint distribution over model outputs of queried points
      batch_size: number of queried points per batch
    #Returns
      Indices of queried points based on their ordering in X_cand
    """
    k_mc_pool_pred = fixed_mask_forward_pass_results.T
    #shape of k_mc_pool_pred is [num_classes, num_candidates, num_masks]
    #Calculating quantities that will be involved in batchbald computations throughout batch assembly
    all_pt_entropies = entropy(k_mc_pool_pred, base=2, axis=0)
    avg_pt_entropies = np.mean(all_pt_entropies, axis=1)
    avg_mc_pool_pred = np.mean(k_mc_pool_pred, axis=2)
    avg_prob_entropies = entropy(avg_mc_pool_pred, base=2, axis=0)
    acq_ind = []
    for itr in range(batch_size):
      #In the first iteration simply compute the BALD objective; no additional MC sampling
      if itr == 0:
        acq_values = avg_prob_entropies - avg_pt_entropies
        acq_ind.append(np.argmax(acq_values))
      else:
        all_acq_values = np.zeros(len(X_cand))
        #y_n-1 sampled from sample over dropout masks ~ probability of y_n-1 | w (i.e. from p(w,y_n-1))
        #Draw batchbald_mc_samples of y hat 1:n-1 
        all_sampled_y = np.zeros((batchbald_mc_samples, itr))
        for sample_itr in range(batchbald_mc_samples):
          mask_ind = np.random.choice(num_masks)
          sampled_y = np.zeros(itr)
          for batch_ind in range(itr):
            prob_vector = k_mc_pool_pred[:, acq_ind[batch_ind], mask_ind]
            sampled_y[batch_ind] = np.random.choice(num_classes, p=prob_vector)
          all_sampled_y[sample_itr,:] = sampled_y
        #Calculating probability of all batchbald_mc_samples y_n-1 sequences under each of the k masks
        p_hat_n_minus_1 = np.zeros((batchbald_mc_samples, num_masks))
        for batchbald_sample_ind in range(batchbald_mc_samples):
          y_n_minus_1 = all_sampled_y[batchbald_sample_ind,:]
          for mask_ind in range(num_masks):
            running_prob = 1.0
            for indiv_acq_ind in range(itr):
              running_prob *= k_mc_pool_pred[int(y_n_minus_1[indiv_acq_ind]), acq_ind[indiv_acq_ind], mask_ind]
            p_hat_n_minus_1[batchbald_sample_ind, mask_ind] = running_prob
        first_term_p_n_minus_1_denom = np.matmul(p_hat_n_minus_1, np.ones((num_masks, 1)))
        #Calculating probability of each new point under each of the k masks (while iterating through candidate points)
        all_joint_entropies = []
        for cand_acq_ind in range(len(X_cand)):
          p_hat_n = k_mc_pool_pred[:, cand_acq_ind,:]
          first_term_numerator = np.matmul(p_hat_n_minus_1, p_hat_n.T)
          first_term = np.divide(first_term_numerator, first_term_p_n_minus_1_denom)
          second_term = np.log2(first_term_numerator/num_masks)
          second_term[np.argwhere(np.isfinite(second_term)==False)] = 0.0
          total_joint_entropy = -1.0 * np.sum(np.multiply(first_term, second_term))/batchbald_mc_samples
          all_joint_entropies.append(total_joint_entropy)
          if cand_acq_ind not in acq_ind:
            curr_batch_ind = acq_ind + [cand_acq_ind]
          else:
            curr_batch_ind = acq_ind
          all_acq_values[cand_acq_ind] = total_joint_entropy - np.sum(avg_pt_entropies[np.array(curr_batch_ind)])
        top_acq_ind = np.flip(np.argsort(all_acq_values))
        found_new_pt = False
        curr_ind = -1
        while found_new_pt == False:
          curr_ind += 1
          if top_acq_ind[curr_ind] not in acq_ind:
            acq_ind.append(top_acq_ind[curr_ind])
            found_new_pt = True
        print('bald curr bs ' + str(itr+1) + ', current acq value: ' + str(all_acq_values[top_acq_ind[curr_ind]]) + ', max joint entropy: ' + str(max(all_joint_entropies)))
    print('function output: ' + str(acq_ind))
    return acq_ind


In [0]:
#Active learning parameters/settings
num_cand = 2000
batch_size = 100
dropout_prob = 0.25
num_experiments = 3
num_acquisitions = 1000 
num_masks = 50
batchbald_mc_samples = 10000
#Keras Model Parameters
num_classes = 10
nb_filters = 40
nb_pool = 2
nb_conv = 4
img_rows = img_cols = 28

In [0]:
#Loading data

data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
		  delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
		 delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.arange(60000)
test_ind = np.arange(10000)

y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=10)
X_test = np.expand_dims(X_test[test_ind], axis=1)
folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Scripts_v1/"
out_folder_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/MultiClass_AL_Results_v1/"

print('Running')	  
all_tr_ind = []
all_acc = []

#Iterating across experiments, each of which begins with a different training set (that is balanced across classes)

for e in range(0,1):
  num_acquisitions = 1000
  acc_file = "BatchBaldAccBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  ind_file = "BatchBaldIndBS"+str(batch_size)+"Ind"+str(e+1)+".npy"
  #Initial training/accuracy
  exp_acc = []
  #exp_acc = list(np.load(out_folder_path+acc_file))
  #train_data_indices = list(np.load(out_folder_path+ind_file))
  train_data_indices = list(np.load(folder_path + 'trainindices' + str(e+1) + '.npy'))
  pool_indices = [i for i in train_ind if i not in train_data_indices]

  #Evaluating initial model
  model_results = run_model(train_data_indices)
  print('trained model')
  exp_acc.append(model_results[0])
  all_acc.append(exp_acc)
  all_tr_ind.append(train_data_indices)
  print('Initial Acc: ' + str(all_acc))
  num_acquisitions = num_acquisitions - batch_size * (len(exp_acc)-1)
  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size):
    cand_indices = np.random.choice(pool_indices, num_cand, replace=False)
    X_cand = np.expand_dims(X_train_All[cand_indices], axis=1)
    conv_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 40,3,3)), p=[dropout_prob, 1-dropout_prob])
    dense_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 1, 128)), p=[dropout_prob, 1-dropout_prob])
    pool_forward_pass_results = fixed_mask_forward_pass(model_results[1], X_cand, num_masks, dropout_prob, conv_masks, dense_masks)  
    #BatchBald acquisition
    batch_ind_ind = batch_bald(pool_forward_pass_results, num_classes, num_masks, batchbald_mc_samples, batch_size)
    all_batch_ind = cand_indices[np.array(batch_ind_ind)]
    for batch_ind in all_batch_ind:
      train_data_indices.append(batch_ind)
      pool_indices.remove(batch_ind)  
    #Retraining and saving results
    model_results = run_model(train_data_indices)
    exp_acc.append(model_results[0])
    all_acc[-1] = exp_acc
    all_tr_ind[-1] = train_data_indices
    np.save(out_folder_path+acc_file, np.array(exp_acc))
    np.save(out_folder_path+ind_file, np.array(train_data_indices))
    print('Exp ' + str(e+1)+', Exp Acc: ' + str(exp_acc) + ', num training points: ' + str(len(train_data_indices)))
    #print('All Ind: ' + str(all_tr_ind))
  print('All Acc: ' + str(all_acc))



Running


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


trained model
Initial Acc: [[0.6186]]
bald curr bs 2, current acq value: 3.436605407238085, max joint entropy: 5.819929131971221
bald curr bs 3, current acq value: 4.524466229656241, max joint entropy: 8.020373886841957
bald curr bs 4, current acq value: 5.11919861650496, max joint entropy: 9.456916481399764
bald curr bs 5, current acq value: 5.433430887231369, max joint entropy: 11.070974722360807
bald curr bs 6, current acq value: 5.514923858698211, max joint entropy: 12.048686580255582
bald curr bs 7, current acq value: 5.604692085987994, max joint entropy: 12.841072061068306
bald curr bs 8, current acq value: 5.631471616031954, max joint entropy: 13.705318915609785
bald curr bs 9, current acq value: 5.576455635314709, max joint entropy: 15.071038893587124
bald curr bs 10, current acq value: 5.60294361126032, max joint entropy: 16.078803082712852
bald curr bs 11, current acq value: 5.747320557064995, max joint entropy: 17.129516800979818
bald curr bs 12, current acq value: 5.6770568

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


Exp 1, Exp Acc: [0.6186, 0.7692], num training points: 130
bald curr bs 2, current acq value: 3.2977220589526235, max joint entropy: 5.803107807593252
bald curr bs 3, current acq value: 4.353912559294691, max joint entropy: 8.038746846762491
bald curr bs 4, current acq value: 4.956021788642195, max joint entropy: 9.94388484324612
bald curr bs 5, current acq value: 5.3237959650523425, max joint entropy: 11.26725555070245
bald curr bs 6, current acq value: 5.481654834790916, max joint entropy: 12.277868054067119
bald curr bs 7, current acq value: 5.536727652802199, max joint entropy: 13.08262916371017
bald curr bs 8, current acq value: 5.651443044939544, max joint entropy: 13.940434537185324
bald curr bs 9, current acq value: 5.601576368880016, max joint entropy: 15.239662595236922
bald curr bs 10, current acq value: 5.64781518155899, max joint entropy: 16.158413148258813
bald curr bs 11, current acq value: 5.697623605417629, max joint entropy: 16.972308641869585
bald curr bs 12, current

...

Exp 1, Exp Acc: [0.6186, 0.7692, 0.8583, 0.8957, 0.9069, 0.9354, 0.9239, 0.9497, 0.9518], num training points: 830
bald curr bs 2, current acq value: 3.286634285951359, max joint entropy: 4.793282345298173
bald curr bs 3, current acq value: 4.357165091178445, max joint entropy: 6.660495018126583
bald curr bs 4, current acq value: 5.019578311369939, max joint entropy: 8.31205462287137
bald curr bs 5, current acq value: 5.345044056803797, max joint entropy: 9.31968671798018
bald curr bs 6, current acq value: 5.494851146605237, max joint entropy: 9.86019400301544
bald curr bs 7, current acq value: 5.619271774130549, max joint entropy: 10.610294486994654
bald curr bs 8, current acq value: 5.6203603851688655, max joint entropy: 11.14218660354383
bald curr bs 9, current acq value: 5.621504459233586, max joint entropy: 11.629537846313404
bald curr bs 10, current acq value: 5.655576400306137, max joint entropy: 12.07590595731483
bald curr bs 11, current acq value: 5.718400650467222, max joint 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


Exp 1, Exp Acc: [0.6186, 0.7692, 0.8583, 0.8957, 0.9069, 0.9354, 0.9239, 0.9497, 0.9518, 0.9458], num training points: 930
bald curr bs 2, current acq value: 3.1329495563498746, max joint entropy: 4.722263094007783
bald curr bs 3, current acq value: 4.181743664942714, max joint entropy: 6.388648314677211
bald curr bs 4, current acq value: 4.881509863736106, max joint entropy: 7.904014422523332
bald curr bs 5, current acq value: 5.261681386560998, max joint entropy: 8.823166674651633
bald curr bs 6, current acq value: 5.479714016878425, max joint entropy: 9.541200465899355
bald curr bs 7, current acq value: 5.592665802634862, max joint entropy: 10.11572739757022
bald curr bs 8, current acq value: 5.614030894308346, max joint entropy: 10.790169199227794
bald curr bs 9, current acq value: 5.636556580844992, max joint entropy: 11.51290351936852
bald curr bs 10, current acq value: 5.678839041565091, max joint entropy: 12.208554579589993
bald curr bs 11, current acq value: 5.67272376219263, 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, kernel_regularizer=<keras.reg...)`


Exp 1, Exp Acc: [0.6186, 0.7692, 0.8583, 0.8957, 0.9069, 0.9354, 0.9239, 0.9497, 0.9518, 0.9458, 0.947], num training points: 1030
All Acc: [[0.6186, 0.7692, 0.8583, 0.8957, 0.9069, 0.9354, 0.9239, 0.9497, 0.9518, 0.9458, 0.947]]
